In [2]:
import os, time
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

#time_str = (datetime.datetime.now()).strftime("%Y%m%d")
#time_str = '20190108'
#xml_dir = '/data/darknet/python/' + time_str + '_xml/'
xml_dir = '/home/yyp/data/selected_xmls/'
SEED_XML_DIR = xml_dir
GENE_XML_DIR = SEED_XML_DIR


# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

#classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']
classes_ = ['person']

for xml_name in seed_xml_names:    
    if(not xml_name.endswith('.xml')):
        continue
    if(not (int(xml_name[-15:-4]) >= 410000)):
        continue   
    #img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    print(SEED_XML_DIR + xml_name)
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)

    root = tree.getroot()
    size = root.find('size')   
    boxes = []
    b = []  
    #wrong_detection_objs = [[393, 408, 52, 83]]
    wrong_detection_objs = [[124, 134, 97, 144]]

    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        for erro_bbox in wrong_detection_objs:
            if(IOU(erro_bbox, b) > 0.4):
                print(IOU(erro_bbox, b))
                print('removed')
                root.remove(obj)
                erro_obj_removed = True
                break
        if(erro_obj_removed):
            continue
        '''
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 or bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue 
        else:
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))
        '''    
    tree.write(GENE_XML_DIR + xml_name)

/home/yyp/data/selected_xmls/00000410001.xml
/home/yyp/data/selected_xmls/00000410002.xml
/home/yyp/data/selected_xmls/00000410003.xml
/home/yyp/data/selected_xmls/00000410004.xml
/home/yyp/data/selected_xmls/00000410005.xml
/home/yyp/data/selected_xmls/00000410006.xml
/home/yyp/data/selected_xmls/00000410007.xml
/home/yyp/data/selected_xmls/00000410008.xml
/home/yyp/data/selected_xmls/00000410009.xml
/home/yyp/data/selected_xmls/00000410010.xml
/home/yyp/data/selected_xmls/00000410011.xml
/home/yyp/data/selected_xmls/00000410012.xml
/home/yyp/data/selected_xmls/00000410013.xml
/home/yyp/data/selected_xmls/00000410014.xml
/home/yyp/data/selected_xmls/00000410015.xml
/home/yyp/data/selected_xmls/00000410016.xml
/home/yyp/data/selected_xmls/00000410017.xml
/home/yyp/data/selected_xmls/00000410018.xml
/home/yyp/data/selected_xmls/00000410019.xml
/home/yyp/data/selected_xmls/00000410020.xml
/home/yyp/data/selected_xmls/00000410021.xml
0.8013937282229965
removed
/home/yyp/data/selected_xmls

/home/yyp/data/selected_xmls/00000410185.xml
/home/yyp/data/selected_xmls/00000410186.xml
/home/yyp/data/selected_xmls/00000410187.xml
0.8380952380952381
removed
/home/yyp/data/selected_xmls/00000410188.xml
0.8380952380952381
removed
/home/yyp/data/selected_xmls/00000410189.xml
0.9375
removed
/home/yyp/data/selected_xmls/00000410190.xml
0.9583333333333334
removed
/home/yyp/data/selected_xmls/00000410191.xml
0.9583333333333334
removed
/home/yyp/data/selected_xmls/00000410192.xml
/home/yyp/data/selected_xmls/00000410193.xml
/home/yyp/data/selected_xmls/00000410194.xml
/home/yyp/data/selected_xmls/00000410195.xml
/home/yyp/data/selected_xmls/00000410196.xml
/home/yyp/data/selected_xmls/00000410197.xml
/home/yyp/data/selected_xmls/00000410198.xml
0.6101694915254238
removed
/home/yyp/data/selected_xmls/00000410199.xml
/home/yyp/data/selected_xmls/00000410200.xml
/home/yyp/data/selected_xmls/00000410201.xml
/home/yyp/data/selected_xmls/00000410202.xml
/home/yyp/data/selected_xmls/00000410203

/home/yyp/data/selected_xmls/00000410494.xml
/home/yyp/data/selected_xmls/00000410495.xml
0.8379888268156425
removed
/home/yyp/data/selected_xmls/00000410496.xml
/home/yyp/data/selected_xmls/00000410497.xml
/home/yyp/data/selected_xmls/00000410498.xml
/home/yyp/data/selected_xmls/00000410499.xml
/home/yyp/data/selected_xmls/00000410500.xml
/home/yyp/data/selected_xmls/00000410501.xml
/home/yyp/data/selected_xmls/00000410502.xml
/home/yyp/data/selected_xmls/00000410503.xml
/home/yyp/data/selected_xmls/00000410504.xml
/home/yyp/data/selected_xmls/00000410505.xml
/home/yyp/data/selected_xmls/00000410506.xml
/home/yyp/data/selected_xmls/00000410507.xml
/home/yyp/data/selected_xmls/00000410508.xml
/home/yyp/data/selected_xmls/00000410509.xml
/home/yyp/data/selected_xmls/00000410510.xml
/home/yyp/data/selected_xmls/00000410511.xml
/home/yyp/data/selected_xmls/00000410512.xml
/home/yyp/data/selected_xmls/00000410513.xml
/home/yyp/data/selected_xmls/00000410514.xml
/home/yyp/data/selected_xmls

/home/yyp/data/selected_xmls/00000410807.xml
/home/yyp/data/selected_xmls/00000410808.xml
/home/yyp/data/selected_xmls/00000410809.xml
/home/yyp/data/selected_xmls/00000410810.xml
/home/yyp/data/selected_xmls/00000410811.xml
/home/yyp/data/selected_xmls/00000410812.xml
/home/yyp/data/selected_xmls/00000410813.xml
/home/yyp/data/selected_xmls/00000410814.xml
/home/yyp/data/selected_xmls/00000410815.xml
/home/yyp/data/selected_xmls/00000410816.xml
/home/yyp/data/selected_xmls/00000410817.xml
/home/yyp/data/selected_xmls/00000410818.xml
/home/yyp/data/selected_xmls/00000410819.xml
/home/yyp/data/selected_xmls/00000410820.xml
/home/yyp/data/selected_xmls/00000410821.xml
/home/yyp/data/selected_xmls/00000410822.xml
/home/yyp/data/selected_xmls/00000410823.xml
/home/yyp/data/selected_xmls/00000410824.xml
/home/yyp/data/selected_xmls/00000410825.xml
/home/yyp/data/selected_xmls/00000410826.xml
/home/yyp/data/selected_xmls/00000410827.xml
/home/yyp/data/selected_xmls/00000410828.xml
/home/yyp/

/home/yyp/data/selected_xmls/00000411094.xml
/home/yyp/data/selected_xmls/00000411095.xml
/home/yyp/data/selected_xmls/00000411096.xml
/home/yyp/data/selected_xmls/00000411097.xml
/home/yyp/data/selected_xmls/00000411098.xml
/home/yyp/data/selected_xmls/00000411099.xml
/home/yyp/data/selected_xmls/00000411100.xml
/home/yyp/data/selected_xmls/00000411101.xml
/home/yyp/data/selected_xmls/00000411102.xml
/home/yyp/data/selected_xmls/00000411103.xml
/home/yyp/data/selected_xmls/00000411104.xml
/home/yyp/data/selected_xmls/00000411105.xml
/home/yyp/data/selected_xmls/00000411106.xml
/home/yyp/data/selected_xmls/00000411107.xml
/home/yyp/data/selected_xmls/00000411108.xml
/home/yyp/data/selected_xmls/00000411109.xml
/home/yyp/data/selected_xmls/00000411110.xml
/home/yyp/data/selected_xmls/00000411111.xml
/home/yyp/data/selected_xmls/00000411112.xml
/home/yyp/data/selected_xmls/00000411113.xml
/home/yyp/data/selected_xmls/00000411114.xml
/home/yyp/data/selected_xmls/00000411115.xml
/home/yyp/

...

/home/yyp/data/selected_xmls/00000411618.xml
/home/yyp/data/selected_xmls/00000411619.xml
/home/yyp/data/selected_xmls/00000411620.xml
/home/yyp/data/selected_xmls/00000411621.xml
/home/yyp/data/selected_xmls/00000411622.xml
/home/yyp/data/selected_xmls/00000411623.xml
/home/yyp/data/selected_xmls/00000411624.xml
/home/yyp/data/selected_xmls/00000411625.xml
/home/yyp/data/selected_xmls/00000411626.xml
/home/yyp/data/selected_xmls/00000411627.xml
/home/yyp/data/selected_xmls/00000411628.xml
/home/yyp/data/selected_xmls/00000411629.xml
/home/yyp/data/selected_xmls/00000411630.xml
/home/yyp/data/selected_xmls/00000411631.xml
/home/yyp/data/selected_xmls/00000411632.xml
/home/yyp/data/selected_xmls/00000411633.xml
/home/yyp/data/selected_xmls/00000411634.xml
/home/yyp/data/selected_xmls/00000411635.xml
/home/yyp/data/selected_xmls/00000411636.xml
/home/yyp/data/selected_xmls/00000411637.xml
/home/yyp/data/selected_xmls/00000411638.xml
/home/yyp/data/selected_xmls/00000411639.xml
/home/yyp/

/home/yyp/data/selected_xmls/00000411921.xml
/home/yyp/data/selected_xmls/00000411922.xml
/home/yyp/data/selected_xmls/00000411923.xml
/home/yyp/data/selected_xmls/00000411924.xml
/home/yyp/data/selected_xmls/00000411925.xml
/home/yyp/data/selected_xmls/00000411926.xml
/home/yyp/data/selected_xmls/00000411927.xml
/home/yyp/data/selected_xmls/00000411928.xml
/home/yyp/data/selected_xmls/00000411929.xml
/home/yyp/data/selected_xmls/00000411930.xml
/home/yyp/data/selected_xmls/00000411931.xml
/home/yyp/data/selected_xmls/00000411932.xml
/home/yyp/data/selected_xmls/00000411933.xml
/home/yyp/data/selected_xmls/00000411934.xml
/home/yyp/data/selected_xmls/00000411935.xml
/home/yyp/data/selected_xmls/00000411936.xml
/home/yyp/data/selected_xmls/00000411937.xml
/home/yyp/data/selected_xmls/00000411938.xml
/home/yyp/data/selected_xmls/00000411939.xml
/home/yyp/data/selected_xmls/00000411940.xml
/home/yyp/data/selected_xmls/00000411941.xml
/home/yyp/data/selected_xmls/00000411942.xml
/home/yyp/

/home/yyp/data/selected_xmls/20190103145920304414_20190103102253.357028_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920334535_20190103102240.526892_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920378933_20190103102240.541478_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920404776_20190103102240.556564_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920433955_20190103102240.613092_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920462727_20190103102240.627198_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920510614_20190103102240.765285_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920561611_20190103102240.777289_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920601852_20190103102240.790760_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920640238_20190103102240.804045_00000410000.xml
/home/yyp/data/selected_xmls/20190103145920675970_20190103102240.817880_00000410000.xml
/home/yyp/data/selected_xmls/201

/home/yyp/data/selected_xmls/20190103151654279023_20190103102253.357028_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806067065_20190103102240.526892_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806208106_20190103102240.541478_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806241856_20190103102240.556564_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806291757_20190103102240.613092_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806336391_20190103102240.627198_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806515415_20190103102240.765285_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806713171_20190103102240.777289_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806835824_20190103102240.790760_00000410000.xml
/home/yyp/data/selected_xmls/20190103151806936494_20190103102240.804045_00000410000.xml
/home/yyp/data/selected_xmls/20190103151807036484_20190103102240.817880_00000410000.xml
/home/yyp/data/selected_xmls/201

/home/yyp/data/selected_xmls/20190103151942116927_20190103102243.373168_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942161383_20190103102243.387198_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942197408_20190103102243.441676_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942253478_20190103102243.469051_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942310488_20190103102243.648537_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942354862_20190103102243.661709_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942428985_20190103102243.699318_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942508142_20190103102243.711801_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942591901_20190103102243.724120_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942681487_20190103102243.888571_00000410000.xml
/home/yyp/data/selected_xmls/20190103151942743570_20190103102243.900797_00000410000.xml
/home/yyp/data/selected_xmls/201